In [29]:
import numpy as np
from pandas import read_csv

In [30]:
#lstm_data = np.genfromtxt('./sample_data/lstm.csv', delimiter=',', skip_header=True)
lstm_data = read_csv('lstm.csv')
lstm_data = lstm_data.drop(['Date'], axis=1)
print(lstm_data)

        Open      High       Low     Close  Adj Close      Volume  \
0   0.137213  0.138747  0.135565  0.137541   0.137541   383506507   
1   0.137523  0.147592  0.137250  0.147503   0.147503   580740990   
2   0.144379  0.157354  0.144031  0.148948   0.148948  1581065491   
3   0.148869  0.150132  0.143649  0.146003   0.146003   898042727   
4   0.145996  0.152199  0.145203  0.151761   0.151761   674961496   
5   0.151779  0.151824  0.146771  0.149095   0.149095   505588732   
6   0.149050  0.149889  0.138071  0.138552   0.138552   721382126   
7   0.138594  0.142097  0.136960  0.138768   0.138768   563817289   
8   0.138764  0.145906  0.138751  0.141206   0.141206   655782652   
9   0.141197  0.141197  0.135852  0.136868   0.136868   490138547   
10  0.136838  0.141345  0.128246  0.128490   0.128490   913773124   
11  0.128488  0.131696  0.125365  0.131553   0.131553   711967119   
12  0.131561  0.135337  0.127846  0.127846   0.127846   587472906   
13  0.127821  0.127851  0.109555  

In [31]:
lstm_data_X = lstm_data.drop(['Close'], axis=1)
lstm_data_y = lstm_data['Close']

In [32]:
look_back = 7 # how many days to look back
batch_size = 5 # size of batches used when training
n_feat = 8 # number of features 
n_target = 1
n_train = 60
n_validation = 5
n_test = 6

In [33]:
lstm_data_X_train = lstm_data_X.iloc[:30,:]
lstm_data_X_val = lstm_data_X.iloc[30:72,:]
lstm_data_X_test = lstm_data_X.iloc[30:72,:]

lstm_data_y_train = lstm_data_y.iloc[:30]
lstm_data_y_val = lstm_data_y.iloc[30:55]
lstm_data_y_test = lstm_data_y.iloc[55:72]
# Convert to numpy arrays
X_train = lstm_data_X_train.to_numpy()
X_val = lstm_data_X_val.to_numpy()
X_test = lstm_data_X_test.to_numpy()
y_train = lstm_data_y_train.to_numpy()
y_val = lstm_data_y_val.to_numpy()
y_test = lstm_data_y_test.to_numpy()

In [34]:
X_test

array([[ 1.22487001e-01,  1.30713001e-01,  1.21361002e-01,
         1.29727006e-01,  9.98922753e+08,  1.22355052e+08,
         5.91602150e-02,  1.40484460e-01],
       [ 1.29721999e-01,  1.40605003e-01,  1.28454998e-01,
         1.36549994e-01,  2.01792681e+09,  2.61769499e+08,
         5.25949730e-02,  1.17735880e-01],
       [ 1.36602998e-01,  1.37274995e-01,  1.28782004e-01,
         1.31013006e-01,  8.82486375e+08,  1.20550284e+08,
        -4.05491660e-02,  1.61422260e-01],
       [ 1.31009996e-01,  1.36494994e-01,  1.29877999e-01,
         1.35867998e-01,  6.10401998e+08,  7.99687633e+07,
         3.70573290e-02,  2.27658000e-01],
       [ 1.35900006e-01,  1.44858003e-01,  1.35702997e-01,
         1.44731998e-01,  1.44501956e+09,  1.96378166e+08,
         6.52397960e-02,  1.69029840e-01],
       [ 1.44724995e-01,  1.52737007e-01,  1.42456993e-01,
         1.42656997e-01,  1.47687551e+09,  2.13740800e+08,
        -1.43368540e-02,  1.53586280e-01],
       [ 1.42556995e-01,  1.485590

In [35]:
y_test

array([], dtype=float64)

In [36]:
import keras
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, ThresholdedReLU, MaxPooling2D, Embedding, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

In [37]:
train_data_gen = TimeseriesGenerator(X_train, y_train, length=look_back, batch_size=batch_size)
val_data_gen = TimeseriesGenerator(X_val, y_val, length=look_back, batch_size=batch_size)
test_data_gen = TimeseriesGenerator(X_test, y_test, length=look_back, batch_size=batch_size)

ValueError: Data and targets have to be of same length. Data length is 17 while target length is 0

In [ ]:
# check generator dimensions
for i in range(len(train_data_gen)):
    x, y = train_data_gen[i]
    print(x.shape, y.shape)

In [ ]:
model_lstm = Sequential()
model_lstm.add(LSTM(32, input_shape=(look_back, n_feat), return_sequences=True))
model_lstm.add(Dropout(0.1))
model_lstm.add(LSTM(32))
model_lstm.add(Dropout(0.1))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer="adam", loss='mse', metrics=["mse"])
print(model_lstm.summary())

In [ ]:
hist = model_lstm.fit_generator(train_data_gen,
                                        steps_per_epoch=10,
                                        epochs=20,
                                        verbose=1,
                                        validation_data=val_data_gen)

In [ ]:
pred = model_lstm.predict(test_data_gen)

In [ ]:
test_data_gen[0]

In [ ]:
pred